In [ ]:
import pandas as pd
import re
from openpyxl import Workbook

def month_number_to_name(month):
    """Convert month number to abbreviated month name."""
    return ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"][month - 1]

def get_monthly_data(sheet, current_month, current_year):
    """Return only the month columns corresponding to the last three months."""
    month_columns = []
    
    last_three_months = []
    for i in range(3):
        month_offset = (current_month - 1 - i)
        month = (month_offset % 12) + 1
        year = current_year + (month_offset // 12)
        last_three_months.append((month, year))
    
    print(f"Last Three Months: {last_three_months}")  # Debugging output

    for col in sheet.columns:
        if re.match(r'M\d{2}', col):
            month_num = int(col[1:3])
            year = int(col[3:])
            if (month_num, year) in last_three_months:
                month_columns.append(col)

    print(f"Identified Month Columns: {month_columns}")  # Debugging output
    return month_columns

def process_paste_sheet(processed_file_path, current_month, current_year):
    # Load the processed file
    xls = pd.ExcelFile(processed_file_path)
    
    # Create a new workbook and the "Paste" sheet
    wb = Workbook()
    ws = wb.active
    ws.title = "Paste"

    # Define the headers for the "Paste" sheet
    headers = ["File Name", "Level 3", "Level 4", "Cost Grouping", "Cost Type", 
               "Finance Region", "Attribute Type", "Period", "Cost", 
               "FTE/Contractor", "Country", "Level 3.5", "Level 4.5", "Entity", 
               "Function", "Mapped Country", "MICA"]
    ws.append(headers)  # Write headers to the first row

    for sheet_name in xls.sheet_names:
        if 'Actual' in sheet_name:
            sheet = pd.read_excel(xls, sheet_name=sheet_name)
            month_columns = get_monthly_data(sheet, current_month, current_year)
            print(f"Processing sheet: {sheet_name}, month columns: {month_columns}")  # Debugging output

            for index, row in sheet.iterrows():
                entity = row.get('Entity', '')
                function = row.get('RTN Level 4', '')
                mica = row.get('Account', '')

                for month_col in month_columns:
                    if month_col.startswith('M'):
                        month_num = int(month_col[1:3])
                        year = int(month_col[3:])
                        period_value = f"{month_number_to_name(month_num)}-{str(year)[-2:]}"  # Format period

                        # Ensure we do not include January 2024 when current is January 2025
                        if not (current_month == 1 and year == 2024):
                            print(f"Inserting into Paste Sheet: {sheet_name}, Period: {period_value}, Entity: {entity}, Function: {function}, MICA: {mica}, Value: {row[month_col]}")
                            
                            if 'DC' in sheet_name:
                                ws.append([sheet_name, '', '', '', '', '', '', period_value, row[month_col], '', '', '', '', entity, function, '', mica])
                            elif 'HC' in sheet_name:
                                ws.append([sheet_name, '', '', '', '', '', '', period_value, '', row[month_col], '', '', '', entity, function, '', mica])
    
    # Save the final workbook
    wb.save('final_output_file.xlsx')

# Example usage
current_month = 1  # January
current_year = 2025
processed_file_path = 'Processed_file.xlsx'  # Update with your file path
process_paste_sheet(processed_file_path, current_month, current_year)
